In [1]:
%load_ext bigdata

In [2]:
%hive_start

In [3]:
%timeout 300

In [4]:
!hadoop fs -rm -f -r tmp;

In [5]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

-rw-r--r--   1 root supergroup       1355 2020-01-20 04:06 /tmp/output/data.tsv


In [6]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 20.016 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.826 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 2.349 seconds


In [7]:
%%hive
--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
SELECT
    concat(key,',',
    count(1))
FROM
    t0
LATERAL VIEW
    explode(c3) t0
GROUP BY 
 key;

--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
OK
Time taken: 0.016 seconds
CREATE TABLE salida
AS
SELECT
    concat(key,',',
    count(1))
FROM
    t0
LATERAL VIEW
    explode(c3) t0
GROUP BY 
 key;
Query ID = root_20200120040656_16e2a5e1-5c0b-4fc8-b140-610a0559ce87
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579492971267_0001, Tracking URL = http://7eba37682953:8088/proxy/application_1579492971267_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579492971267_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-20 04:07

In [8]:
%%hive
DROP TABLE IF EXISTS tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.027 seconds


In [9]:
%%hive
-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120040802_934b13c1-8ff9-451f-b1ac-304d9ceff2fe
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579492971267_0002, Tracking URL = http://7eba37682953:8088/proxy/application_1579492971267_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579492971267_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 04:08:20,119 Stage-1 map = 0%,  reduce = 0%
2020-01-20 04:08:28,473 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.81 sec
MapReduce Total cumulative CPU time: 2 seconds 810 msec
Ended Job = job_1579492971267_0002
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000

In [10]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [11]:
%hive_quit